In [1]:
import keras
import pandas as pd
from glob import glob
import os
import cv2
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

preprocess_path = "F:/DR_detection/preprocessedtrain2"
train_files = glob(os.path.join(preprocess_path, "*.jpeg"))
input_path = "F:\DR_detection"
df = pd.read_csv(os.path.join(input_path, "trainLabels.csv"))

C:\Users\dell\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import imgaug as ia
from imgaug import augmenters as iaa

seq = iaa.Sequential(
    [
        iaa.Fliplr(0.5), 
        iaa.Affine(
            scale=(0.9, 1.1),
            rotate=(-180, 180), 
        ),
    ],
    random_order=True
)

In [ ]:
def show_augmented_images(img_file):
    img = cv2.imread(img_file)
    imgs = [img] + seq.augment_images([img for x in range(7)])

    plt.figure(figsize=(16, 6))
    for i in range(0, 4):
        plt.subplot(1, 4, i + 1)
        plt.imshow(imgs[i])
        
show_augmented_images(train_files[0])
show_augmented_images(train_files[1])
show_augmented_images(train_files[2])
show_augmented_images(train_files[3])

In [6]:
def get_level(file, df):
    basename = os.path.basename(file)
    image_id = basename.split(".")[0]

    mini_df = df[df['image'] == image_id]
    if len(mini_df) < 1:
        return None

    return mini_df.values[0][1]

# Example: 1 -> [0, 1, 0, 0, 0]
def get_onehot(level):
    level_vec = np.zeros(5)
    level_vec[level] = 1
    
    return level_vec

In [ ]:
from keras.utils.data_utils import Sequence

class DataSequence(Sequence):
    def __init__(self, file_list, batch_size, augment=False):
        self.file_list = file_list
        self.batch_size = batch_size
        self.augment = augment

    def __len__(self):
        return len(self.file_list) // self.batch_size

    def __getitem__(self, idx):
        imgs = []
        labels = []
        
        i = idx * self.batch_size
        
        while len(imgs) < self.batch_size:
            file = self.file_list[i]
            
            level = get_level(file, df)
            if level is None:
                print("missing level: " + file)
                i += 1
                continue
            
            img = cv2.imread(file)
            
            if img is not None:
                imgs.append(img)
                labels.append(get_onehot(level))
                #class_weights= [0:0.01, 1:0.2,2:0.220,3:0.270,4:0.3]
                #class_weights = get_class_weights(current_epoch)

            i += 1

        # Image augmentation
        #if self.augment:
        #    imgs = seq.augment_images(imgs)

        return np.asarray(imgs).astype(np.float16), np.asarray(labels).astype(np.int8)

In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from datetime import datetime as dt
def get_experiment_id():
    time_str = dt.now().strftime('%Y-%m-%d-%H-%M-%S')
    experiment_id = 'base_{}'.format(time_str)

    return experiment_id
def get_model(input_dim):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', recent:///c65efa87dac3a53d6d4d7de55c806e19))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    #model.add(Conv2D(64, (3, 3), activation='relu'))
    #model.add(Conv2D(64, (3, 3), activation='relu'))
    
    #model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(96, (3, 3), activation='relu'))
    model.add(Conv2D(96, (3, 3), activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(192, (3, 3), activation='relu'))
    model.add(Conv2D(192, (3, 3), activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.003, momentum=0.9, nesterov=True), metrics=['accuracy'])

    return model

In [9]:
model = get_model(256)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 96)      27744     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 122, 122, 96)      83040     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 61, 96)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 59, 59, 128)       110720    
__________

In [10]:
train_files = glob("F:\DR_detection\preprocessedtrain\*jpeg")

n_val_files = len(train_files) // 10

val_file_list = train_files[:n_val_files]
train_file_list = train_files[n_val_files:]

In [11]:
train_files

['F:\\DR_detection\\preprocessedtrain\\10003_left.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10003_right.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10007_left.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10007_right.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10009_left.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10009_right.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\1000_left.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\1000_right.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10010_left.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10010_right.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10013_left.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10013_right.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10014_left.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10014_right.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10015_left.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10015_right.jpeg',
 'F:\\DR_detection\\preprocessedtrain\\10017_left.jpeg',
 'F:\\DR_detection\\prepr

In [ ]:
batch_size = 16

train_gen = DataSequence(train_file_list, batch_size, augment=True)
validate_gen = DataSequence(val_file_list, batch_size, augment=True)

model = get_model(256)

experiment_id = get_experiment_id()

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1),
    ModelCheckpoint(experiment_id + "-val_loss_checkpoint.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, mode='auto', epsilon=0.01, cooldown=0, min_lr=1e-6)
]

history = model.fit_generator(generator=train_gen, 
                              validation_data=validate_gen,
                              steps_per_epoch=len(train_gen),
                              validation_steps=len(validate_gen),
                              verbose=1,
                              epochs=125,
                              callbacks=callbacks,
                              workers=7, # recommend: number of cpu cores
                              use_multiprocessing=True)

C:\Users\dell\Anaconda3\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/125


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\dell\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\dell\Anaconda3\lib\site-packages\keras\utils\data_utils.py", line 565, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\dell\Anaconda3\lib\site-packages\keras\utils\data_utils.py", line 548, in <lambda>
    initargs=(seqs,))
  File "C:\Users\dell\Anaconda3\lib\multiprocessing\context.py", line 119, in Pool
    context=self.get_context())
  File "C:\Users\dell\Anaconda3\lib\multiprocessing\pool.py", line 174, in __init__
    self._repopulate_pool()
  File "C:\Users\dell\Anaconda3\lib\multiprocessing\pool.py", line 239, in _repopulate_pool
    w.start()
  File "C:\Users\dell\Anaconda3\lib\multiprocessing\process.py", line 105, in start
    self._popen = self.